In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils import data
from torch.utils.data import Dataset, DataLoader

In [2]:
def get_finetune_resnet():
    # 加载预训练的 ResNet50
    model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)

    # 替换最后一层
    in_feature = model.fc.in_features
    model.fc = nn.Sequential(
    nn.Linear(in_feature, 256), nn.BatchNorm1d(256), nn.ReLU(),
    nn.Linear(256, 176)
    )

    return model

In [3]:
import pandas as pd
train_df = pd.read_csv("./dataset/leaf/train.csv")

all_labels = sorted(set(train_df['label']))
label2id = {label: idx for idx, label in enumerate(all_labels)}
id2label = {v: k for k, v in label2id.items()}
print("mapping: ", label2id)

train_df['label_id'] = train_df['label'].map(label2id)
train_df

mapping:  {'abies_concolor': 0, 'abies_nordmanniana': 1, 'acer_campestre': 2, 'acer_ginnala': 3, 'acer_griseum': 4, 'acer_negundo': 5, 'acer_palmatum': 6, 'acer_pensylvanicum': 7, 'acer_platanoides': 8, 'acer_pseudoplatanus': 9, 'acer_rubrum': 10, 'acer_saccharinum': 11, 'acer_saccharum': 12, 'aesculus_flava': 13, 'aesculus_glabra': 14, 'aesculus_hippocastamon': 15, 'aesculus_pavi': 16, 'ailanthus_altissima': 17, 'albizia_julibrissin': 18, 'amelanchier_arborea': 19, 'amelanchier_canadensis': 20, 'amelanchier_laevis': 21, 'asimina_triloba': 22, 'betula_alleghaniensis': 23, 'betula_jacqemontii': 24, 'betula_lenta': 25, 'betula_nigra': 26, 'betula_populifolia': 27, 'broussonettia_papyrifera': 28, 'carpinus_betulus': 29, 'carpinus_caroliniana': 30, 'carya_cordiformis': 31, 'carya_glabra': 32, 'carya_ovata': 33, 'carya_tomentosa': 34, 'castanea_dentata': 35, 'catalpa_bignonioides': 36, 'catalpa_speciosa': 37, 'cedrus_atlantica': 38, 'cedrus_deodara': 39, 'cedrus_libani': 40, 'celtis_occiden

,image,label,label_id
0,images/0.jpg,maclura_pomifera,78
1,images/1.jpg,maclura_pomifera,78
2,images/2.jpg,maclura_pomifera,78
3,images/3.jpg,maclura_pomifera,78
4,images/4.jpg,maclura_pomifera,78
...,...,...,...
18348,images/18348.jpg,aesculus_glabra,14
18349,images/18349.jpg,liquidambar_styraciflua,76
18350,images/18350.jpg,cedrus_libani,40
18351,images/18351.jpg,prunus_pensylvanica,125


In [4]:
from PIL import Image
import os
class LeavesDataset(Dataset):
    def __init__(self, image_path, labels, root_dir, trans):
        super().__init__()
        self.data_dir = root_dir
        self.image_path_list = image_path
        self.labels = labels
        self.transform = trans
    
    def __len__(self):
        return len(self.image_path_list)
    
    def __getitem__(self, idx):
        image_idx_path = os.path.join(self.data_dir, self.image_path_list[idx])
        image = self.transform(Image.open(image_idx_path).convert("RGB"))
        label = self.labels[idx]
        return image, label
        

In [5]:
# from tqdm import tqdm
# def train(model, lr, epochs, train_loader, test_loader, writer, fold_i):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     # device = torch.device("cpu")
#     model = model.to(device)
#     global_step = 0
#     test_step = 0
#     optimer = torch.optim.Adam(model.parameters(), lr=lr)
#     criterion = torch.nn.CrossEntropyLoss()

#     best_acc = 0.0
    
#     for epoch in tqdm(range(epochs)):
#         train_right = 0
#         test_right = 0
#         train_num = 0
#         test_num = 0
#         model.train()
#         for X, y in train_loader:
#             X, y = X.to(device), y.to(device)
#             y_hat = model(X)
#             l = criterion(y_hat, y)
#             optimer.zero_grad()
#             l.backward()
#             optimer.step()
#             train_right += (y==y_hat.argmax(dim=1)).sum().item()
#             total_num += len(y)

#             writer.add_scalar(f"fold_{fold_i}/loss/train", l.item(), global_step)
#             global_step += 1
            

#         with torch.no_grad():
#             model.eval()
#             for X, y in test_loader:
#                 X, y = X.to(device), y.to(device)
#                 y_hat = model(X)
#                 l = criterion(y_hat, y)
#                 test_right += (y==y_hat.argmax(dim=1)).sum().item()

#                 writer.add_scalar(f"fold_{fold_i}/loss/test", l.item(), test_step)
#                 test_step += 1
        
#         writer.add_scalar(f"fold_{fold_i}/accuracy/train", train_right/total_num, epoch)
#         writer.add_scalar(f"fold_{fold_i}/accuracy/test", test_right/total_num, epoch)
#         test_acc = test_right/total_num
#         if test_acc > best_acc:
#             best_acc = test_acc
#             torch.save(model.state_dict(), f"./models/best_model_fold{fold_i}.ckpt")
#             print(f"Best model saved for fold {fold_i} at epoch {epoch+1} with Val Acc {test_acc:.4f}")

In [6]:
from tqdm import tqdm
def train(model, lr, epochs, train_loader, test_loader, writer, fold_i):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    optimizer = torch.optim.AdamW(
        [{'params': [param for name, param in model.named_parameters() if 'fc.' not in name]},
        {'params': model.fc.parameters(), 'lr': 1e-4 * 10}],
        lr=1e-4, weight_decay=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    global_step = 0
    test_step = 0
    
    best_acc = 0.0

    for epoch in range(epochs):
        print(f"epoch: {epoch+1}/{epochs}")
        model.train()
        train_num = train_right = test_num = test_right = 0
        for X, y in tqdm(train_loader):
            X, y = X.to(device), y.to(device)
            y_hat = model(X)
            l = criterion(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            writer.add_scalar(f"fold_{fold_i}/loss/train", l.item(), global_step)
            global_step += 1
            train_num += len(y)
            train_right += (y==y_hat.argmax(dim=1)).sum().item()
        
        with torch.no_grad():
            model.eval()
            for X, y in test_loader:
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                l = criterion(y_hat, y)
                test_right += (y==y_hat.argmax(dim=1)).sum().item()
                test_num += len(y)

                writer.add_scalar(f"fold_{fold_i}/loss/test", l.item(), test_step)
                test_step += 1
    
        test_acc = test_right/test_num
        print(f"fold_{fold_i}/accuracy/train: {train_right/train_num}")
        print(f"fold_{fold_i}/aaccuracy/test: {test_right/test_num}")
        writer.add_scalars(f"fold_{fold_i}/acc", {'train': train_right/train_num, 'test': test_right/test_num}, epoch)
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f"./models/best_model_fold{fold_i}.ckpt")
            print(f"Best model saved for fold {fold_i} at epoch {epoch+1} with Val Acc {test_acc:.4f}")

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir="./log/leaves")

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss()

kf = KFold(n_splits=3, shuffle=True, random_state=42)

train_trans = torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.RandomRotation(degrees=90),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomVerticalFlip(),
            # torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            # torchvision.transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.95, 1.05)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

test_trans = torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
    print(f"Fold {fold + 1}")

    model = get_finetune_resnet()

    t_df = train_df.iloc[train_idx].reset_index(drop=True)
    v_df = train_df.iloc[val_idx].reset_index(drop=True)

    train_dataset = LeavesDataset(
        image_path=t_df['image'].tolist(),
        labels=t_df['label_id'].tolist(),
        root_dir="./dataset/leaf",
        trans=train_trans
    )

    val_dataset = LeavesDataset(
        image_path=v_df['image'].tolist(),
        labels=v_df['label_id'].tolist(),
        root_dir="./dataset/leaf",
        trans=test_trans
    )

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=8)


    train(model=model, lr=0.001, epochs=10, train_loader=train_loader, test_loader=val_loader, writer=writer, fold_i=fold+1)
    # model.load_state_dict(torch.load("./models/leaves_best_model_fold1.ckpt"))
    # test_right = 0
    # test_num = 0
    # model.to(device)
    # with torch.no_grad():
    #     model.eval()
    #     for X, y in val_loader:
    #         X, y = X.to(device), y.to(device)
    #         y_hat = model(X)
    #         l = criterion(y_hat, y)
    #         test_right += (y==y_hat.argmax(dim=1)).sum().item()
    #         test_num += len(y)
    # print(test_right/test_num)
    
    





Fold 1
epoch: 1/10


100%|██████████| 48/48 [00:21<00:00,  2.19it/s]


fold_1/accuracy/train: 0.29579076420106254
fold_1/aaccuracy/test: 0.4883949002942138
Best model saved for fold 1 at epoch 1 with Val Acc 0.4884
epoch: 2/10


100%|██████████| 48/48 [00:20<00:00,  2.29it/s]


fold_1/accuracy/train: 0.6912954638332652
fold_1/aaccuracy/test: 0.7366786531546257
Best model saved for fold 1 at epoch 2 with Val Acc 0.7367
epoch: 3/10


...

fold_1/accuracy/train: 0.9770331017572538
fold_1/aaccuracy/test: 0.9344557044785877
epoch: 10/10


100%|██████████| 48/48 [00:21<00:00,  2.28it/s]


fold_1/accuracy/train: 0.9822639967306906
fold_1/aaccuracy/test: 0.933801896044459
Fold 2
epoch: 1/10


 54%|█████▍    | 26/48 [00:12<00:10,  2.10it/s]


KeyboardInterrupt: 